In [1]:
!pip install psycopg2-binary
!pip install psycopg2

  Using cached psycopg2_binary-2.9.3-cp38-cp38-win_amd64.whl (1.1 MB)


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] 拒绝访问。: 'D:\\00Tools\\Anaconda3\\Lib\\site-packages\\psycopg2\\_psycopg.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import psycopg2, os

print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host="localhost",
    port='5433',
    database="ceo",
    user="postgres",
    password="psql")

Connecting to the PostgreSQL database...


In [3]:
# create a cursor
cur = conn.cursor()

In [4]:
# execute a statement
print('PostgreSQL database version:')
cur.execute('SELECT version()')

# display the PostgreSQL database server version
db_version = cur.fetchone()
print(db_version)

PostgreSQL database version:
('PostgreSQL 12.4, compiled by Visual C++ build 1914, 64-bit',)


In [5]:
cur.execute("drop table if exists company")
cur.execute("drop table if exists ceo")
cur.execute("drop table if exists city")

conn.commit()

In [6]:
createCmd = """ CREATE TABLE city (
                city VARCHAR(30) PRIMARY KEY,
                State  VARCHAR(20),
                country   CHAR(20)
                )
            """
    
cur.execute(createCmd)
conn.commit()

In [7]:
cur.execute("INSERT into city VALUES('Mobile', 'AL', 'USA')")
cur.execute("INSERT into city VALUES('West Godavari', 'Andhra-Pradesh', 'India')")
cur.execute("INSERT into city VALUES('Scarsdale', 'NY', 'USA')")
cur.execute("INSERT into city VALUES('Royal Oak', 'MI', 'USA')")
cur.execute("INSERT into city VALUES('Kumamoto', '', 'Japan')")
cur.execute("INSERT into city VALUES('Shanghai','', 'China')")
cur.execute("INSERT into city VALUES('Rennes', 'Illes-et-Villaine', 'France')")
cur.execute("INSERT into city VALUES('Cupertino', 'AL', 'USA')")
cur.execute("INSERT into city VALUES('Armonk', 'NY', 'USA')")
cur.execute("INSERT into city VALUES('Seattle', 'WA', 'USA')")
cur.execute("INSERT into city VALUES('Detroit', 'MI', 'USA')")
cur.execute("INSERT into city VALUES('Tokyo', '', 'Japan')")
cur.execute("INSERT into city VALUES('Hangzhou', 'Zhejiang', 'China')")
cur.execute("INSERT into city VALUES('Paris', '', 'France')")
conn.commit()

In [8]:
createCmd = """ CREATE TABLE CEO (
                ceo_id SERIAL PRIMARY KEY,
                firstname VARCHAR(50),
                lastname  VARCHAR(50),
                birth_yr  int,
                birth_city  VARCHAR(30),
                become_CEO  int,
                FOREIGN KEY (birth_city) REFERENCES city(city)
                )
            """
    
cur.execute(createCmd)
conn.commit()

In [9]:
cur.execute("INSERT into CEO VALUES(1, 'Tim','Cook', 1960, 'Mobile', 2011)")
cur.execute("INSERT into CEO VALUES(2, 'Arvind', 'Krishna', 1962, 'West Godavari', 2020)")
cur.execute("INSERT into CEO VALUES(3, 'Andy', 'Jassy', 1968, 'Scarsdale', 2021)")
cur.execute("INSERT into CEO VALUES(4, 'Marry', 'Barra', 1961, 'Royal Oak', 2014)")
cur.execute("INSERT into CEO VALUES(5, 'Kenichiro', 'Yoshida', 1959, 'Kumamoto', 2018)")
cur.execute("INSERT into CEO VALUES(6, 'Daniel', 'Zhang', 1972, 'Shanghai', 2015)")
cur.execute("INSERT into CEO VALUES(7, 'François-Henri', 'Pinault', 1962, 'Rennes', 2005)")
conn.commit()

In [10]:
createCmd = """ CREATE TABLE company (
                company_id SERIAL PRIMARY KEY,
                name        VARCHAR(30) NOT NULL,
                HQ          VARCHAR(30),
                ceo_id      int,
                found_yr    int,
                FOREIGN KEY (HQ) REFERENCES city(city),
                FOREIGN KEY (ceo_id) REFERENCES CEO(ceo_id)
                )
            """
    
cur.execute(createCmd)
conn.commit()

In [11]:
cur.execute("INSERT into company VALUES(1, 'Apple, Inc.', 'Cupertino',1, 1976)")
cur.execute("INSERT into company VALUES(2, 'IBM', 'Armonk',2,1911)")
cur.execute("INSERT into company VALUES(3, 'Amazon','Seattle',3, 1994)")
cur.execute("INSERT into company VALUES(4, 'General Motors', 'Detroit',4,1908)")
cur.execute("INSERT into company VALUES(5, 'Sony Corporation','Tokyo',5, 1946)")
cur.execute("INSERT into company VALUES(6, 'Alibaba Group', 'Hangzhou',6,1999)")
cur.execute("INSERT into company VALUES(7, 'Kering SA', 'Paris', 7, 1963)")
conn.commit()

In [12]:
# What’s the age of the company whose CEO was born in the state of Michigan?
queryCmd = '''SELECT company.name, extract(year from now())-found_yr from CEO 
join CITY on CEO.birth_city=CITY.city 
join company on company.ceo_id=CEO.ceo_id
where CITY.state='MI'  
'''
cur.execute(queryCmd)    
print("The number of rows: ", cur.rowcount)

rows = cur.fetchall()
for row in rows:
    print("company name:", row[0], ", age of the company", row[1])
   

The number of rows:  1
company name: General Motors , age of the company 114.0


In [13]:
# What are the full names of CEOs of companies founded in the USA before 1946?
queryCmd = '''SELECT concat(firstname,' ', lastname) fullname from CEO
join company on company.ceo_id=CEO.ceo_id 
join CITY on CITY.city=company.HQ
where city.country = 'USA' and company.found_yr < 1946'''
cur.execute(queryCmd)    
print("The number of rows: ", cur.rowcount)

rows = cur.fetchall()
# print(rows)
for row in rows:
    print("full name:", row[0])


The number of rows:  2
full name: Arvind Krishna
full name: Marry Barra


In [14]:
# Which companies have CEOs younger than 55?
queryCmd = '''SELECT company.name from CEO
join company on company.ceo_id=CEO.ceo_id 
where extract(year from now())-CEO.birth_yr<55'''   
cur.execute(queryCmd)    
print("The number of rows: ", cur.rowcount)

rows = cur.fetchall()

for row in rows:
    print("company name:", row[0])


The number of rows:  2
company name: Amazon
company name: Alibaba Group


In [15]:
# close the communication with the PostgreSQL
cur.close()